<a href="https://colab.research.google.com/github/FKz11/NLP/blob/main/lesson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson-4

Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами
https://disk.yandex.ru/i/v5HM-ENiGXZVpQ
https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

### Библиотеки

In [46]:
!pip -qq install pymorphy2

In [47]:
!pip -qq install stop_words

In [51]:
!pip -qq install -U gensim

     |████████████████████████████████| 24.1 MB 60.2 MB/s 


In [ ]:
!pip -qq install pyLDAvis

In [76]:
import pandas as pd
import numpy as np

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from gensim import corpora
from gensim.models import ldamodel

from tqdm import tqdm

from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

### Данные

In [1]:
!wget "https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv"

--2022-08-31 13:49:19--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-08-31 13:49:19--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf9d31863835e4496697a179a40.dl.dropboxusercontent.com/cd/0/inline/BsDEkKs4OeP-0kqDB8XLrKdFIwKXTxtJ0VXbYWN6yf79WGrN3m4Bplcyj-7dYPGtD5DfbY3ldAYgcIyMeAiUOnMicpMiAheMFDS3ddjyZstf6ugLRS9KIlEZLgeuOWRSwXiAsJRN1sr8QJynXNek1ckLN4T8Ok5jmqp529JXBrRc-A/file# [following]
--2022-08-31 13:49:19--  https://ucf9d31863835e4496697a179a40.dl.dropboxusercontent.com/cd/0/inline/BsDEkKs4OeP-0kqDB8XLrKdFIwKXTxtJ0VXbYWN6yf79WGrN3m4Bplcyj-7dYPGtD5DfbY3ldAYgcIyM

In [2]:
!wget "https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv"

--2022-08-31 13:51:04--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2022-08-31 13:51:05--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbccf7aac1fcbab29d42ad0f2a4.dl.dropboxusercontent.com/cd/0/inline/BsCxDxXUWpulW5iROsJeZtVYXXb6rH9OVS9MgLNzLpBBsYtb4vBjLprGzqP7sXm60v7KQZovzKFM4YMRBBsBxnYfoTeIJiQrA-YizC1w51NOmfkhLl1R91RW2WpH608EkNFysudXdaC-FbnRoxqMKxhavw10vOk3kx_XfHPViexq0w/file# [following]
--2022-08-31 13:51:05--  https://ucbccf7aac1fcbab29d42ad0f2a4.dl.dropboxusercontent.com/cd/0/inline/BsCxDxXUWpulW5iROsJeZtVYXXb6rH9OVS9MgLNzLpBBsYtb4vBjLprGzqP7sXm60v7KQZovzKFM4YMR

In [13]:
pos = pd.read_csv('positive.csv', sep=';', names=list(range(12)))
neg = pd.read_csv('negative.csv', sep=';', names=list(range(12)))

In [14]:
pos.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


Нас интересует только текст

In [35]:
df = pd.concat([pos[[3]], neg[[3]]]).rename({3: 'text'}, axis=1)
df.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


### Предобработка

In [49]:
def preprocess_txt(text, morpher, sw, exclude):
    spls = "".join(i for i in text.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw]
    return spls

In [50]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + [''] + [' '])
exclude = set(string.punctuation)

for text in tqdm(df['text']):
  spls = preprocess_txt(text, morpher, sw, exclude)
  sentences.append(spls)

100%|██████████| 226834/226834 [12:37<00:00, 299.55it/s]


Уберём специфичные стоп слова конкретно для этих данных

In [134]:
my_sw = ['rt']
sentences = [[w for w in text if w not in my_sw] for text in sentences]

### Модель LDA

Сперва создадим словарь

In [135]:
dictionary = corpora.Dictionary(sentences)

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')

Векторизуем твиты:

In [136]:
corpus = [dictionary.doc2bow(text) for text in sentences]
corpora.MmCorpus.serialize('tweet.model', corpus) 

Теперь можем обучать модель:

In [163]:
%%time 

lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=5, chunksize=10000, update_every=1, passes=4)

CPU times: user 3min 52s, sys: 5.99 s, total: 3min 58s
Wall time: 3min 54s


### Метрики

In [170]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.0001996157364985286


In [205]:
sentences = [i for i in sentences if len(i) > 0]
coherence_model_lda = CoherenceModel(model=lda, texts=sentences, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

Средняя когерентность:  0.28557227496967275


### Посмотрим на получившиеся темы:

In [169]:
lda.show_topics(num_words=30, formatted=True)

[(0,
  '0.021*"любить" + 0.017*"знать" + 0.014*"скучать" + 0.013*"свой" + 0.011*"большой" + 0.010*"написать" + 0.009*"—" + 0.006*"вк" + 0.006*"самый" + 0.006*"любимый" + 0.006*"оо" + 0.006*"фотка" + 0.006*"ненавидеть" + 0.005*"говорить" + 0.005*"грустно" + 0.005*"забыть" + 0.005*"твиттер" + 0.005*"хороший" + 0.004*"мама" + 0.004*"думать" + 0.004*"любовь" + 0.004*"целый" + 0.004*"чувство" + 0.004*"парень" + 0.004*"oo" + 0.004*"блин" + 0.004*"маленький" + 0.004*"слово" + 0.004*"smampgt" + 0.003*"увидеть"'),
 (1,
  '0.023*"делать" + 0.014*"блин" + 0.012*"вообще" + 0.011*"пойти" + 0.009*"d" + 0.009*"знать" + 0.009*"урок" + 0.008*"школа" + 0.008*"бля" + 0.008*"сидеть" + 0.008*"понимать" + 0.006*"изз" + 0.006*"завтра" + 0.006*"писать" + 0.006*"простить" + 0.006*"заболеть" + 0.006*"учить" + 0.005*"че" + 0.005*"бесить" + 0.004*"нормально" + 0.004*"класс" + 0.004*"учиться" + 0.004*"гулять" + 0.004*"хватать" + 0.004*"сука" + 0.004*"дом" + 0.004*"смотреть" + 0.004*"ужас" + 0.004*"обидно" + 0.004*

Опишем темы сами

In [167]:
topics = {0: 'любовь', 1:'школа', 2:'прогулка', 3:'фильмы', 4:'деятельность'}

Где-то они не совсем очевидны, но тематическое разделение безусловно произошло.

Стоит отметить, что определеню темы на 80% способствуют сущиствительные и на 15% глаголы, поэтому в теории для описания тем можно остовлять только их, отфильтровывая каким-то образом всё остольное.

### Визуализация

In [168]:
%%time 

vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

CPU times: user 35.7 s, sys: 1.11 s, total: 36.9 s
Wall time: 38.3 s


### Интерпретация

Видно, что темы разделились очень хорошо: они не пересекаются и находяться на большом растоянии друг от друга. Из этого можно сделать интуитивный вывод, что с реальным числом глобальных тем мы близки, так как если бы их было например меньше, то темы бы сильно пересекались друг с другом.

Наведясь на кружок слева, справа можно посмотреть на то, как часто слово используется в данной теме, относительно общего использовния. Так же это характеризует, как сильно слово описывает выбранную тему.